In [1]:
import os
import sklearn
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

np.random.seed(42)
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"
warnings.filterwarnings(action='ignore', category=UserWarning)
sklearn.set_config(print_changed_only=True)

In [2]:
train_path = 'input/inputTrain.csv'
test_path = 'input/inputTest.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [3]:
train_df.drop(columns='Unnamed: 0', inplace=True)
test_df.drop(columns='Unnamed: 0', inplace=True)
train_df.T

,0,1,2,3,4,5,6,7,8,9,...,29989,29990,29991,29992,29993,29994,29995,29996,29997,29998
carid,1.000000,2.000000,5.000000,6.000000,7.000000,10.000000,12.000000,13.000000,14.000000,18.000000,...,64261.000000,42866.000000,24598.000000,72533.000000,27237.000000,18531.000000,4294.000000,20584.000000,11490.000000,28672.000000
tradeTime,0.840336,0.827731,0.802521,0.844538,0.848739,0.756303,0.789916,0.920168,0.974790,0.718487,...,0.802521,-0.655462,0.873950,0.235294,0.546218,0.701681,-0.798319,-0.310924,-0.130252,-0.012605
mileage,-0.534415,0.315772,1.028558,-0.089701,-0.154414,0.810883,0.082312,-0.379565,1.083441,-0.204128,...,-0.637535,-0.402953,0.100099,0.056000,-0.136970,0.538139,0.680581,0.478198,0.441610,-0.558113
color_1,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
color_2,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
l6,0.668317,-0.049505,-0.222772,1.074257,0.574257,-0.198020,0.222772,-0.148515,-0.143564,-0.173267,...,0.985149,0.841584,-0.074257,0.970297,1.014851,0.396040,-0.049505,0.099010,-0.198020,-0.168317
l7,-1.175993,-0.271037,-0.715401,-0.235116,-0.813323,-1.280552,-1.730886,-0.130919,-0.499289,-0.334701,...,0.454206,0.105606,-0.430253,0.416118,0.167502,-1.196486,0.693357,-0.568485,0.480517,0.405306
l8,-0.247441,-0.252904,-0.604505,0.635244,-0.169895,-0.937574,-0.769392,-0.273900,-0.481734,-0.475091,...,0.941090,0.705644,-0.333004,0.940380,0.847939,-0.369935,0.270279,-0.047498,0.040985,-0.002290
l9,0.229515,-0.226296,-0.531781,0.918661,0.311352,-0.627546,-0.475458,-0.273810,-0.352787,-0.290792,...,1.062477,0.741570,-0.313537,1.007418,0.964963,-0.112968,0.009225,-0.362698,-0.199016,-0.164260


In [4]:
train_X = train_df.drop(columns=['carid', 'price'])
test_X = test_df.drop(columns=['carid', 'price'])
train_y = train_df['price']
test_y = test_df['price']

In [5]:
# def criterion(pred_y, label):
#     pred = np.exp(pred_y) - 1
#     y = np.exp(label) - 1
#     dy = np.abs(pred - y) / y
#     mape = np.sum(dy) / len(pred)
#     acc = len(dy[dy <= 0.05]) / len(pred)
#     return 0.2 * (1 - mape) + 0.8 * acc

def criterion(pred_y, label):
    pred = np.exp(pred_y) - 1
    y = np.exp(label) - 1
    dy = np.abs(pred - y) / y
    mape = np.sum(dy) / len(pred)
    return mape

def mape(y_true, y_pred):
    pred = np.exp(y_pred) - 1
    y = np.exp(y_true) - 1
    return np.mean(np.abs((pred - y) / y))

In [6]:
# score_criterion = make_scorer(criterion, greater_is_better=False)
score_criterion = make_scorer(mape, greater_is_better=False)

In [7]:
cv_X_train, cv_X_test, cv_y_train, cv_y_test = train_test_split(train_X, train_y, random_state = 42, train_size = 0.9)

# tpot = TPOTRegressor(generations=5, verbosity=3, scoring='neg_mean_absolute_error', random_state=42)
tpot = TPOTRegressor(generations=5, verbosity=3, scoring=score_criterion, random_state=42)
# tpot = TPOTRegressor(generations=5, verbosity=3, random_state=42)
tpot.fit(cv_X_train, cv_y_train)
print(tpot.score(cv_X_test, cv_y_test))
tpot.export('pipeline.py')

30 operators have been imported by TPOT.
Skipped pipeline #50 due to time out. Continuing to the next pipeline.           
Skipped pipeline #58 due to time out. Continuing to the next pipeline.           
Skipped pipeline #76 due to time out. Continuing to the next pipeline.           
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 The 'loss' parameter of GradientBoostingRegressor must be a str among {'absolute_error', 'squared_error', 'quantile', 'huber'}. Got 'ls' instead..
_